<a href="https://colab.research.google.com/github/fdac25/football/blob/main/football.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [308]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import kagglehub

In [309]:
# Import the dataset. Too big to include in GitHub so have to download it to colab

# Download latest version
path = kagglehub.dataset_download("philiphyde1/nfl-stats-1999-2022") # To access just use path variable

print("Path to dataset files:", path)

!dir {path}

Path to dataset files: C:\Users\chris\.cache\kagglehub\datasets\philiphyde1\nfl-stats-1999-2022\versions\17
 Volume in drive C is OS
 Volume Serial Number is 0C1F-C8E2

 Directory of C:\Users\chris\.cache\kagglehub\datasets\philiphyde1\nfl-stats-1999-2022\versions\17

11/11/2025  04:33 PM    <DIR>          .
11/11/2025  04:33 PM    <DIR>          ..
11/11/2025  04:33 PM        65,567,697 weekly_player_stats_defense.csv
11/11/2025  04:33 PM       105,635,176 weekly_player_stats_offense.csv
11/11/2025  04:33 PM         1,711,446 weekly_team_stats_defense.csv
11/11/2025  04:33 PM         3,692,917 weekly_team_stats_offense.csv
11/11/2025  04:33 PM        13,455,015 yearly_player_stats_defense.csv
11/11/2025  04:33 PM        27,959,466 yearly_player_stats_offense.csv
11/11/2025  04:33 PM           190,956 yearly_team_stats_defense.csv
11/11/2025  04:33 PM           450,662 yearly_team_stats_offense.csv
               8 File(s)    218,663,335 bytes
               2 Dir(s)  279,323,226,112 b

In [310]:
#Display the first few rows of the dataset to understand its structure and contents.
df_yearly_player_stats = pd.read_csv(path + "/yearly_player_stats_offense.csv")
df_yearly_player_stats.head()

,player_id,player_name,position,birth_year,draft_year,draft_round,draft_pick,draft_ovr,height,weight,...,delta_comp_pct,delta_int_pct,delta_pass_td_pct,delta_ypa,delta_yptarget,delta_ypr,delta_rush_td_pct,delta_ypc,delta_td_pct,delta_yptouch
0,00-0000865,Charlie Batch,QB,1974,1998.0,2.0,30.0,60.0,74.0,216.0,...,0.220036,-1.0,-1.0,0.025641,0.000000,0.000000,0.0,0.000000,-0.909091,-0.388795
1,00-0004541,Donald Driver,WR,1975,1999.0,7.0,7.0,213.0,72.0,194.0,...,0.000000,NaN,NaN,0.000000,-0.172028,-0.068732,0.0,0.000000,49.000000,1.226852
2,00-0006101,Tony Gonzalez,TE,1976,1997.0,1.0,13.0,13.0,77.0,247.0,...,0.000000,0.0,0.0,0.000000,0.273859,-0.216170,0.0,0.000000,3.833333,0.236242
3,00-0006101,Tony Gonzalez,TE,1976,1997.0,1.0,13.0,13.0,77.0,247.0,...,0.000000,NaN,NaN,0.000000,-0.165038,0.085776,0.0,0.000000,-0.413793,0.085776
4,00-0007091,Matt Hasselbeck,QB,1975,1998.0,6.0,34.0,187.0,76.0,235.0,...,0.149281,-1.0,-1.0,-0.162698,0.000000,0.000000,-1.0,-0.337868,0.500000,0.250509


In [311]:
df_yearly_team_stats = pd.read_csv(path + "/yearly_team_stats_defense.csv")
df_yearly_team_stats.head()

,team,season,season_type,safety,interception,fumble,fumble_lost,fumble_forced,fumble_not_forced,fumble_out_of_bounds,...,average_tackle_with_assist,average_sack,average_qb_hit,average_def_touchdown,average_defensive_two_point_attempt,average_defensive_two_point_conv,average_defensive_extra_point_attempt,average_defensive_extra_point_conv,rush_pct,pass_pct
0,ARI,2012,REG,0.0,22.0,19.0,11.0,14.0,6.0,2.0,...,2.625000,2.375000,5.250000,0.250000,0.0,0.0,0.0,0.0,0.47,0.51
1,ATL,2012,POST,0.0,1.0,2.0,2.0,2.0,0.0,0.0,...,1.388889,1.777778,4.111111,0.111111,0.0,0.0,0.0,0.0,0.49,0.51
2,ATL,2012,REG,0.0,20.0,16.0,11.0,15.0,1.0,2.0,...,1.388889,1.777778,4.111111,0.111111,0.0,0.0,0.0,0.0,0.41,0.59
3,BAL,2012,POST,0.0,6.0,6.0,4.0,5.0,1.0,0.0,...,8.050000,2.300000,5.400000,0.150000,0.0,0.0,0.0,0.0,0.40,0.59
4,BAL,2012,REG,0.0,13.0,22.0,10.0,8.0,14.0,3.0,...,8.050000,2.300000,5.400000,0.150000,0.0,0.0,0.0,0.0,0.45,0.55


In [312]:
# Filter the dataframe to keep only the columns we need (Only year 2024, those not POST season, and things to calculate our score)
df_yearly_player_stats_filtered = df_yearly_player_stats.loc[(df_yearly_player_stats['season'] == 2024) & (df_yearly_player_stats['season_type'] != 'POST'),
                                                             ['player_name', 'position', 'season_complete_pass', 'season_pass_attempts', 'season_incomplete_pass', 'season_passing_yards',
                                                              'season_receiving_yards', 'season_rush_attempts', 'season_rushing_yards', 'season_fantasy_points_ppr', 'games_played_season',
                                                              'season_rush_touchdown', 'season_pass_touchdown', 'season_interception', 'season_fumble', 'season_fumble_lost',
                                                              'season_receptions', 'season_targets', 'season_receiving_touchdown'
                                                             ]]

# Examples
players = ['Amon-Ra St. Brown', 'Lamar Jackson', 'Dak Prescott', 'George Kittle', 'Sam LaPorta', 'Justin Jefferson', 'Bijan Robinson', 'Saquon Barkley', 'Cameron Dicker']
df_selected_players = df_yearly_player_stats_filtered[df_yearly_player_stats_filtered['player_name'].isin(players)]
df_selected_players

,player_name,position,season_complete_pass,season_pass_attempts,season_incomplete_pass,season_passing_yards,season_receiving_yards,season_rush_attempts,season_rushing_yards,season_fantasy_points_ppr,games_played_season,season_rush_touchdown,season_pass_touchdown,season_interception,season_fumble,season_fumble_lost,season_receptions,season_targets,season_receiving_touchdown
6630,Dak Prescott,QB,185.0,278.0,93.0,1978.0,0.0,13.0,54.0,137.52,8,1.0,11.0,8.0,4.0,1.0,0.0,0.0,0.0
6642,George Kittle,TE,0.0,0.0,0.0,0.0,939.0,0.0,0.0,176.90,12,0.0,4.0,1.0,0.0,0.0,61.0,71.0,4.0
6724,Lamar Jackson,QB,316.0,470.0,154.0,4172.0,0.0,140.0,915.0,505.38,17,4.0,41.0,4.0,10.0,5.0,0.0,0.0,0.0
6732,Saquon Barkley,RB,0.0,0.0,0.0,0.0,278.0,348.0,2005.0,348.30,16,13.0,2.0,0.0,2.0,1.0,33.0,43.0,2.0
6814,Justin Jefferson,WR,1.0,1.0,0.0,22.0,1533.0,1.0,3.0,307.48,17,0.0,10.0,5.0,1.0,0.0,103.0,149.0,10.0
6878,Amon-Ra St. Brown,WR,1.0,1.0,0.0,7.0,1263.0,2.0,6.0,327.18,17,0.0,15.0,1.0,1.0,1.0,115.0,140.0,14.0
6978,Bijan Robinson,RB,0.0,0.0,0.0,0.0,431.0,305.0,1456.0,339.70,17,14.0,1.0,0.0,1.0,0.0,61.0,72.0,1.0
7041,Sam LaPorta,TE,0.0,0.0,0.0,0.0,726.0,0.0,0.0,170.60,16,0.0,7.0,2.0,0.0,0.0,60.0,81.0,7.0


In [313]:
# Explore the positions we have in our dataset
print("Unique positions in dataset:")
unique_positions = df_yearly_player_stats_filtered['position'].unique()
print(unique_positions)

print("\nPlayers per position:")
position_counts = df_yearly_player_stats_filtered['position'].value_counts()
print(position_counts)

Unique positions in dataset:
['QB' 'TE' 'P' 'FB' 'WR' 'RB' 'SS' 'OLB' 'CB' 'DE' 'DT' 'FS' 'MLB']

Players per position:
position
WR     188
RB     109
TE     103
QB      74
FB       6
P        5
SS       5
CB       5
FS       2
OLB      1
DE       1
DT       1
MLB      1
Name: count, dtype: int64


In [314]:
# Clean data - removing players with missing data

# Remove any rows that don't have a player name, position, or games played
df_yearly_player_stats_filtered = df_yearly_player_stats_filtered.dropna(subset=(['player_name', 'position', 'games_played_season']))


# Replace any missing numbers with 0 (means they didn't do that stat)
numeric_columns = df_yearly_player_stats_filtered.select_dtypes(include=[np.number]).columns
df_yearly_player_stats_filtered[numeric_columns] = df_yearly_player_stats_filtered[numeric_columns].fillna(0)

# Reset the index after dropping rows
df_yearly_player_stats_filtered = df_yearly_player_stats_filtered.reset_index(drop=True)

print("\nAfter cleaning Players per position:")
position_counts = df_yearly_player_stats_filtered['position'].value_counts()
print(position_counts)


After cleaning Players per position:
position
WR     188
RB     109
TE     103
QB      74
FB       6
P        5
SS       5
CB       5
FS       2
OLB      1
DE       1
DT       1
MLB      1
Name: count, dtype: int64


In [ ]:
# Only focusing on the main offensive positions, combining FB with RB
offensive_positions = ['QB', 'RB', 'WR', 'TE', 'FB']

df_yearly_player_stats_filtered = df_yearly_player_stats_filtered[df_yearly_player_stats_filtered['position'].isin(offensive_positions)]

# combine the fb positions into the rb column
df_yearly_player_stats_filtered['position'] = df_yearly_player_stats_filtered['position'].replace('FB', 'RB')

# Filter to players who played at least half the season (8 games)
df_yearly_player_stats_filtered = df_yearly_player_stats_filtered[df_yearly_player_stats_filtered['games_played_season'] >= 8]

print(f"Total offensive position players: {len(df_yearly_player_stats_filtered)}")
print(df_yearly_player_stats_filtered['position'].value_counts())

Total offensive position players: 324
position
WR    132
RB     86
TE     67
QB     39
Name: count, dtype: int64


In [316]:
# Setting up the scoring for positions
allQBs = (df_yearly_player_stats_filtered['position'] == 'QB')
qbsAboveThresh = (df_yearly_player_stats_filtered['season_passing_yards'] > 1500)


# Filter so we only include quarter backs that have at least 1500 passing yards
qbs = df_yearly_player_stats_filtered[allQBs & qbsAboveThresh].copy()

# Shows that we got rid of 4 QBs
print(qbs.shape)

# Passing stats for a QB
qbs['completion_percent'] = (qbs['season_complete_pass'] / qbs['season_pass_attempts']) * 100
qbs['yards_per_attempt'] = qbs['season_passing_yards'] / qbs['season_pass_attempts']
qbs['touchdown_percent'] = (qbs['season_pass_touchdown'] / qbs['season_pass_attempts']) * 100
qbs['interception_percent'] = (qbs['season_interception'] / qbs['season_pass_attempts']) * 100
# will use fumbles stats too but already exists so dont need to make column
print(qbs[['player_name', 'completion_percent', 'yards_per_attempt', 'touchdown_percent', 'interception_percent', 'season_fumble_lost']].head())

(35, 19)
         player_name  completion_percent  yards_per_attempt  \
0      Aaron Rodgers           64.223386           6.801047   
4   Matthew Stafford           66.797642           7.390963   
7     Russell Wilson           64.652568           7.498489   
8       Kirk Cousins           69.336384           8.027460   
19        Geno Smith           72.291297           7.673179   

    touchdown_percent  interception_percent  season_fumble_lost  
0            4.886562              1.919721                 2.0  
4            3.929273              1.571709                 2.0  
7            4.833837              1.510574                 4.0  
8            4.118993              3.661327                 2.0  
19           3.730018              2.664298                 0.0  


In [317]:
# we decided to use z score because it standardizes the data making it easier to compare to one another
qbs['completion_zScore'] = (qbs['completion_percent'] - qbs['completion_percent'].mean()) / qbs['completion_percent'].std()
qbs['yards_per_attempt_zScore'] = (qbs['yards_per_attempt'] - qbs['yards_per_attempt'].mean()) / qbs['yards_per_attempt'].std()
qbs['touchdown_zScore'] = (qbs['touchdown_percent'] - qbs['touchdown_percent'].mean()) / qbs['touchdown_percent'].std()
qbs['interception_zScore'] = -1 * ((qbs['interception_percent'] - qbs['interception_percent'].mean()) / qbs['interception_percent'].std())

# Fumbles as raw count (flip sign because fewer is better)
qbs['fumbles_zScore'] = -1 * ((qbs['season_fumble_lost'] - qbs['season_fumble_lost'].mean()) / qbs['season_fumble_lost'].std())

# Average the z-scores for overall passing rating
qbs['overall_QB_rating'] = (qbs['completion_zScore'] + qbs['yards_per_attempt_zScore'] + qbs['touchdown_zScore'] + qbs['interception_zScore'] + qbs['fumbles_zScore']) / 5

print("\nTop 10 QBs:")
print(qbs.nlargest(10, 'overall_QB_rating')[['player_name', 'completion_percent', 'yards_per_attempt', 'touchdown_percent', 'interception_percent', 'overall_QB_rating']])


Top 10 QBs:
        player_name  completion_percent  yards_per_attempt  touchdown_percent  \
66       Jared Goff           74.003795           8.783681           7.210626   
140   Lamar Jackson           67.234043           8.876596           8.723404   
149  Baker Mayfield           73.465704           8.122744           7.400722   
23       Derek Carr           68.978102           7.828467           5.474453   
492  Jayden Daniels           70.276008           7.575372           5.307856   
213     Jordan Love           64.734300           8.185990           6.038647   
205  Tua Tagovailoa           74.293785           7.437853           5.084746   
239      Joe Burrow           71.539658           7.648523           6.687403   
224  Justin Herbert           66.267465           7.724551           4.590818   
150      Josh Allen           64.360587           7.821803           5.870021   

     interception_percent  overall_QB_rating  
66               2.277040           1.364545  
1

In [318]:
# same thing for rb

# Setting up the scoring for positions
allRBs = (df_yearly_player_stats_filtered['position'] == 'RB')
rbsAboveThresh = (df_yearly_player_stats_filtered['season_rushing_yards'] > 300)

# Filter so we only include running backs that have at least 500 rushing yards
rbs = df_yearly_player_stats_filtered[allRBs & rbsAboveThresh].copy()

# Shows that we got rid of 
print(rbs.shape)

# Calculate rushing stats
rbs['yards_per_carry'] = rbs['season_rushing_yards'] / rbs['season_rush_attempts']
rbs['rush_touchdown_rate'] = (rbs['season_rush_touchdown'] / rbs['season_rush_attempts']) * 100
rbs['fumble_rate'] = (rbs['season_fumble_lost'] / rbs['season_rush_attempts']) * 100
rbs['receiving_yards_per_game'] = rbs['season_receiving_yards'] / rbs['games_played_season']

print(rbs[['player_name', 'yards_per_carry', 'rush_touchdown_rate', 'fumble_rate', 'receiving_yards_per_game']].head())

(51, 19)
       player_name  yards_per_carry  rush_touchdown_rate  fumble_rate  \
46  Ameer Abdullah         4.712121             3.030303     1.515152   
56   Derrick Henry         5.910769             5.230769     0.307692   
74     Aaron Jones         4.462745             1.960784     1.176471   
86    James Conner         4.577406             3.765690     0.418410   
93   Austin Ekeler         4.797297             5.405405     0.000000   

    receiving_yards_per_game  
46                 20.076923  
56                 11.352941  
74                 24.000000  
86                 25.875000  
93                 31.454545  


In [319]:
# Calculate z-scores for rushing
rbs['yards_per_carry_zScore'] = (rbs['yards_per_carry'] - rbs['yards_per_carry'].mean()) / rbs['yards_per_carry'].std()
rbs['touchdown_rate_zScore'] = (rbs['rush_touchdown_rate'] - rbs['rush_touchdown_rate'].mean()) / rbs['rush_touchdown_rate'].std()
rbs['total_yards_zScore'] = (rbs['season_rushing_yards'] - rbs['season_rushing_yards'].mean()) / rbs['season_rushing_yards'].std()
rbs['fumble_zScore'] = -1 * ((rbs['fumble_rate'] - rbs['fumble_rate'].mean()) / rbs['fumble_rate'].std())
rbs['receiving_yards_zScore'] = (rbs['receiving_yards_per_game'] - rbs['receiving_yards_per_game'].mean()) / rbs['receiving_yards_per_game'].std()

# Average the z-scores for overall rushing rating
rbs['overall_RB_rating'] = (rbs['yards_per_carry_zScore'] + rbs['touchdown_rate_zScore'] + rbs['total_yards_zScore'] + rbs['fumble_zScore'] + rbs['receiving_yards_zScore']) / 5

print("\nTop 10 RBs:")
print(rbs.nlargest(10, 'overall_RB_rating')[['player_name', 'season_rushing_yards', 'yards_per_carry', 'rush_touchdown_rate', 'receiving_yards_per_game', 'overall_RB_rating']])


Top 10 RBs:
        player_name  season_rushing_yards  yards_per_carry  \
426    Jahmyr Gibbs                1412.0         5.648000   
56    Derrick Henry                1921.0         5.910769   
147  Saquon Barkley                2005.0         5.761494   
361  Bijan Robinson                1456.0         4.773770   
295      James Cook                1009.0         4.874396   
441    Bucky Irving                1122.0         5.420290   
93    Austin Ekeler                 355.0         4.797297   
414    Devon Achane                 907.0         4.467980   
104    Alvin Kamara                 950.0         4.166667   
86     James Conner                1094.0         4.577406   

     rush_touchdown_rate  receiving_yards_per_game  overall_RB_rating  
426             6.400000                 28.000000           1.366669  
56              5.230769                 11.352941           1.192801  
147             3.735632                 17.375000           1.143024  
361             

In [320]:
# add them back to main df
df_yearly_player_stats_filtered['overall_QB_rating'] = 0.0
df_yearly_player_stats_filtered['overall_RB_rating'] = 0.0

# Update with calculated ratings
for idx in qbs.index:
    df_yearly_player_stats_filtered.loc[idx, 'overall_QB_rating'] = qbs.loc[idx, 'overall_QB_rating']

for idx in rbs.index:
    df_yearly_player_stats_filtered.loc[idx, 'overall_RB_rating'] = rbs.loc[idx, 'overall_RB_rating']

df_yearly_player_stats_filtered.head()

,player_name,position,season_complete_pass,season_pass_attempts,season_incomplete_pass,season_passing_yards,season_receiving_yards,season_rush_attempts,season_rushing_yards,season_fantasy_points_ppr,...,season_rush_touchdown,season_pass_touchdown,season_interception,season_fumble,season_fumble_lost,season_receptions,season_targets,season_receiving_touchdown,overall_QB_rating,overall_RB_rating
0,Aaron Rodgers,QB,368.0,573.0,205.0,3897.0,0.0,22.0,107.0,306.58,...,0.0,28.0,11.0,5.0,2.0,0.0,0.0,0.0,-0.128084,0.0
4,Matthew Stafford,QB,340.0,509.0,169.0,3762.0,0.0,30.0,41.0,252.58,...,0.0,20.0,8.0,6.0,2.0,0.0,0.0,0.0,0.083751,0.0
7,Russell Wilson,QB,214.0,331.0,117.0,2482.0,0.0,43.0,155.0,200.78,...,2.0,16.0,5.0,5.0,4.0,0.0,0.0,0.0,-0.052014,0.0
8,Kirk Cousins,QB,303.0,437.0,134.0,3508.0,0.0,23.0,0.0,210.32,...,0.0,18.0,16.0,13.0,2.0,0.0,0.0,0.0,-0.014840,0.0
10,Kyle Juszczyk,RB,0.0,0.0,0.0,0.0,200.0,5.0,26.0,54.60,...,1.0,2.0,1.0,1.0,1.0,19.0,30.0,2.0,0.000000,0.0


In [321]:
# Receiving rating for WRs
# Setting up the scoring for positions
allWRs = (df_yearly_player_stats_filtered['position'] == 'WR')
wrsAboveThresh = (df_yearly_player_stats_filtered['season_receiving_yards'] > 500)
wrsAboveRecThresh = (df_yearly_player_stats_filtered['season_receptions'] > 48)

# Filter so we only include running backs that have at least 500 receiving yards
wideRec = df_yearly_player_stats_filtered[allWRs & wrsAboveThresh & wrsAboveRecThresh].copy()

# Shows how many we got rid of
print(wideRec.shape)

# Calculate receiving stats
wideRec['yards_per_reception'] = wideRec['season_receiving_yards'] / wideRec['season_receptions']
wideRec['catch_rate'] = (wideRec['season_receptions'] / wideRec['season_targets']) * 100
wideRec['touchdown_rate'] = (wideRec['season_receiving_touchdown'] / wideRec['season_receptions']) * 100
wideRec['yards_per_game'] = wideRec['season_receiving_yards'] / wideRec['games_played_season']

print(wideRec[['player_name', 'yards_per_reception', 'catch_rate', 'touchdown_rate', 'yards_per_game']].head())

(49, 21)
      player_name  yards_per_reception  catch_rate  touchdown_rate  \
13   Keenan Allen            10.628571   57.851240       10.000000   
26  Davante Adams            12.456790   61.363636        9.876543   
27  Davante Adams            12.456790   61.363636        9.876543   
28  Davante Adams            12.456790   61.363636        9.876543   
29     Mike Evans            13.567568   70.476190       14.864865   

    yards_per_game  
13       49.600000  
26       77.615385  
27       77.615385  
28       77.615385  
29       71.714286  


In [322]:
# Calculate z-scores for receiving
wideRec['yards_per_reception_zScore'] = (wideRec['yards_per_reception'] - wideRec['yards_per_reception'].mean()) / wideRec['yards_per_reception'].std()
wideRec['catch_rate_zScore'] = (wideRec['catch_rate'] - wideRec['catch_rate'].mean()) / wideRec['catch_rate'].std()
wideRec['touchdown_rate_zScore'] = (wideRec['touchdown_rate'] - wideRec['touchdown_rate'].mean()) / wideRec['touchdown_rate'].std()
wideRec['yards_per_game_zScore'] = (wideRec['yards_per_game'] - wideRec['yards_per_game'].mean()) / wideRec['yards_per_game'].std()
wideRec['fumbles_zScore'] = -1 * ((wideRec['season_fumble_lost'] - wideRec['season_fumble_lost'].mean()) / wideRec['season_fumble_lost'].std())

# Average the z-scores for overall receiving rating
wideRec['overall_WR_rating'] = (wideRec['yards_per_reception_zScore'] + wideRec['catch_rate_zScore'] + wideRec['touchdown_rate_zScore'] + wideRec['yards_per_game_zScore'] + wideRec['fumbles_zScore']) / 5

print("\nTop 10 WRs:")
print(wideRec.nlargest(10, 'overall_WR_rating')[['player_name', 'season_receptions', 'yards_per_reception', 'catch_rate', 'touchdown_rate', 'overall_WR_rating']])


Top 10 WRs:
           player_name  season_receptions  yards_per_reception  catch_rate  \
263      Ja'Marr Chase              127.0            13.448819   75.595238   
189         A.J. Brown               67.0            16.104478   70.526316   
218   Justin Jefferson              103.0            14.883495   69.127517   
29          Mike Evans               74.0            13.567568   70.476190   
488       Brian Thomas               87.0            14.735632   67.968750   
266      DeVonta Smith               64.0            12.437500   80.000000   
494      Ladd McConkey               63.0            14.539683   71.590909   
408     Jordan Addison               63.0            13.888889   64.948454   
425         Puka Nacua               72.0            12.277778   75.000000   
271  Amon-Ra St. Brown              115.0            10.982609   82.142857   

     touchdown_rate  overall_WR_rating  
263       13.385827           1.251006  
189       10.447761           0.992001  
218  

In [323]:
# Receiving rating for TEs (same calculations as WRs)
# Setting up the scoring for positions
allTEs = (df_yearly_player_stats_filtered['position'] == 'TE')
tesAboveThresh = (df_yearly_player_stats_filtered['season_receiving_yards'] > 300)
tesAboveRecThresh = (df_yearly_player_stats_filtered['season_receptions'] > 40)

# Filter so we only include running backs that have at least 500 receiving yards
tightEnds = df_yearly_player_stats_filtered[allTEs & tesAboveThresh & tesAboveRecThresh].copy()

# Shows how many we got rid of
print(tightEnds.shape)

# Calculate receiving stats
tightEnds['yards_per_reception'] = tightEnds['season_receiving_yards'] / tightEnds['season_receptions']
tightEnds['catch_rate'] = (tightEnds['season_receptions'] / tightEnds['season_targets']) * 100
tightEnds['touchdown_rate'] = (tightEnds['season_receiving_touchdown'] / tightEnds['season_receptions']) * 100
tightEnds['yards_per_game'] = tightEnds['season_receiving_yards'] / tightEnds['games_played_season']

print(tightEnds[['player_name', 'yards_per_reception', 'catch_rate', 'touchdown_rate', 'yards_per_game']].head())

(27, 21)
      player_name  yards_per_reception  catch_rate  touchdown_rate  \
12      Zach Ertz             9.909091   74.157303       10.606061   
15   Travis Kelce             8.484536   75.193798        3.092784   
51  Austin Hooper            10.577778   78.947368        6.666667   
65   Hunter Henry            10.212121   68.750000        3.030303   
73  George Kittle            15.393443   85.915493        6.557377   

    yards_per_game  
12       38.470588  
15       51.437500  
51       28.000000  
65       42.125000  
73       78.250000  


In [324]:
# Calculate z-scores for TEs
tightEnds['yards_per_reception_zScore'] = (tightEnds['yards_per_reception'] - tightEnds['yards_per_reception'].mean()) / tightEnds['yards_per_reception'].std()
tightEnds['catch_rate_zScore'] = (tightEnds['catch_rate'] - tightEnds['catch_rate'].mean()) / tightEnds['catch_rate'].std()
tightEnds['touchdown_rate_zScore'] = (tightEnds['touchdown_rate'] - tightEnds['touchdown_rate'].mean()) / tightEnds['touchdown_rate'].std()
tightEnds['yards_per_game_zScore'] = (tightEnds['yards_per_game'] - tightEnds['yards_per_game'].mean()) / tightEnds['yards_per_game'].std()
tightEnds['fumbles_zScore'] = -1 * ((tightEnds['season_fumble_lost'] - tightEnds['season_fumble_lost'].mean()) / tightEnds['season_fumble_lost'].std())

# Average the z-scores for overall receiving rating
tightEnds['overall_TE_rating'] = (tightEnds['yards_per_reception_zScore'] + tightEnds['catch_rate_zScore'] + tightEnds['touchdown_rate_zScore'] + tightEnds['yards_per_game_zScore'] + tightEnds['fumbles_zScore']) / 5

print("\nTop 10 TEs:")
print(tightEnds.nlargest(10, 'overall_TE_rating')[['player_name', 'season_receptions', 'yards_per_reception', 'catch_rate', 'touchdown_rate', 'overall_TE_rating']])


Top 10 TEs:
       player_name  season_receptions  yards_per_reception  catch_rate  \
73   George Kittle               61.0            15.393443   85.915493   
135   Mark Andrews               55.0            12.236364   80.882353   
421    Sam LaPorta               60.0            12.100000   74.074074   
436   Brock Bowers              112.0            10.660714   75.675676   
409   Tucker Kraft               50.0            14.140000   72.463768   
324   Trey McBride              111.0            10.324324   76.027397   
216      Cole Kmet               47.0            10.085106   87.037037   
96     Jonnu Smith               88.0            10.045455   79.279279   
12       Zach Ertz               66.0             9.909091   74.157303   
272     Kyle Pitts               47.0            12.808511   67.142857   

     touchdown_rate  overall_TE_rating  
73         6.557377           1.606199  
135       20.000000           1.084283  
421       11.666667           0.586093  
436     

In [325]:
# add them back to main df
df_yearly_player_stats_filtered['overall_WR_rating'] = 0.0
df_yearly_player_stats_filtered['overall_TE_rating'] = 0.0

# Update with calculated ratings
for idx in wideRec.index:
    df_yearly_player_stats_filtered.loc[idx, 'overall_WR_rating'] = wideRec.loc[idx, 'overall_WR_rating']

for idx in tightEnds.index:
    df_yearly_player_stats_filtered.loc[idx, 'overall_TE_rating'] = tightEnds.loc[idx, 'overall_TE_rating']

df_yearly_player_stats_filtered.head()

,player_name,position,season_complete_pass,season_pass_attempts,season_incomplete_pass,season_passing_yards,season_receiving_yards,season_rush_attempts,season_rushing_yards,season_fantasy_points_ppr,...,season_interception,season_fumble,season_fumble_lost,season_receptions,season_targets,season_receiving_touchdown,overall_QB_rating,overall_RB_rating,overall_WR_rating,overall_TE_rating
0,Aaron Rodgers,QB,368.0,573.0,205.0,3897.0,0.0,22.0,107.0,306.58,...,11.0,5.0,2.0,0.0,0.0,0.0,-0.128084,0.0,0.0,0.0
4,Matthew Stafford,QB,340.0,509.0,169.0,3762.0,0.0,30.0,41.0,252.58,...,8.0,6.0,2.0,0.0,0.0,0.0,0.083751,0.0,0.0,0.0
7,Russell Wilson,QB,214.0,331.0,117.0,2482.0,0.0,43.0,155.0,200.78,...,5.0,5.0,4.0,0.0,0.0,0.0,-0.052014,0.0,0.0,0.0
8,Kirk Cousins,QB,303.0,437.0,134.0,3508.0,0.0,23.0,0.0,210.32,...,16.0,13.0,2.0,0.0,0.0,0.0,-0.014840,0.0,0.0,0.0
10,Kyle Juszczyk,RB,0.0,0.0,0.0,0.0,200.0,5.0,26.0,54.60,...,1.0,1.0,1.0,19.0,30.0,2.0,0.000000,0.0,0.0,0.0
